In [2]:
# -*- coding: utf-8 -*-

#import wave
import sys
import random
import math
import os     #
import pyaudio
from socket import *
from random import * 
import numpy 
from scipy.signal import blackmanharris, fftconvolve
from numpy import argmax, sqrt, mean, diff, log
from matplotlib.mlab import find

# See http://www.swharden.com/blog/2013-05-09-realtime-fft-audio-visualization-with-python/
class SoundRecorder:
        
    def __init__(self):
        self.RATE=44100
        self.BUFFERSIZE=3072 #1024 is a good buffer size 3072 works for Pi
        self.secToRecord=.05
        self.threadsDieNow=False
        self.newAudio=False
        
    def setup(self):
        self.buffersToRecord=int(self.RATE*self.secToRecord/self.BUFFERSIZE)
        if self.buffersToRecord==0: self.buffersToRecord=1
        self.samplesToRecord=int(self.BUFFERSIZE*self.buffersToRecord)
        self.chunksToRecord=int(self.samplesToRecord/self.BUFFERSIZE)
        self.secPerPoint=1.0/self.RATE
        self.p = pyaudio.PyAudio()
        #dong = "/Users/zosia/Documents/Processing/examples/learningprocessing2/examples/chp20_sound/files/dingdong.wav"
        #wf = wave.open(dong, 'rb')
        # open wav file - DEBUG ONLY
        #self.inStream = self.p.open(format=p.get_format_from_width(wf.getsampwidth()),
        #        channels=wf.getnchannels(),
        #        rate=wf.getframerate(),
        #        output=True)
        
        # Uncomment this for real data
        self.inStream = self.p.open(format=pyaudio.paInt16,channels=1,rate=self.RATE,input=True,frames_per_buffer=self.BUFFERSIZE)
        self.xsBuffer=numpy.arange(self.BUFFERSIZE)*self.secPerPoint
        self.xs=numpy.arange(self.chunksToRecord*self.BUFFERSIZE)*self.secPerPoint
        self.audio=numpy.empty((self.chunksToRecord*self.BUFFERSIZE),dtype=numpy.int16)               
    
    def close(self):
        self.p.close(self.inStream)
    
    def getAudio(self):
        audioString=self.inStream.read(self.BUFFERSIZE)
        self.newAudio=True
        return numpy.fromstring(audioString,dtype=numpy.int16)
        
# See https://github.com/endolith/waveform-analyzer/blob/master/frequency_estimator.py
def parabolic(f, x): 
    xv = 1/2. * (f[x-1] - f[x+1]) / (f[x-1] - 2 * f[x] + f[x+1]) + x
    yv = f[x] - 1/4. * (f[x-1] - f[x+1]) * (xv - x)
    return (xv, yv)
    
# See https://github.com/endolith/waveform-analyzer/blob/master/frequency_estimator.py
def freq_from_autocorr(raw_data_signal, fs):                          
    corr = fftconvolve(raw_data_signal, raw_data_signal[::-1], mode='full')
    corr = corr[len(corr)/2:]
    d = diff(corr)
    start = find(d > 0)[0]
    peak = argmax(corr[start:]) + start
    px, py = parabolic(corr, peak)
    return fs / px    

def loudness(chunk):
    data = numpy.array(chunk, dtype=float) / 32768.0
    ms = math.sqrt(numpy.sum(data ** 2.0) / len(data))
    if ms < 10e-8: ms = 10e-8
    return 10.0 * math.log(ms, 10.0)
        


def find_nearest(array, value):
    index = (numpy.abs(array - value)).argmin()
    return array[index]

def closest_value_index(array, guessValue):
    # Find closest element in the array, value wise
    closestValue = find_nearest(array, guessValue)
    # Find indices of closestValue
    indexArray = numpy.where(array==closestValue)
    # Numpys 'where' returns a 2D array with the element index as the value
    return indexArray[0][0]

def build_default_tuner_range():
    
    return {65.41:'C2', 
            69.30:'C2#',
            73.42:'D2',  
            77.78:'E2b', 
            82.41:'E2',  
            87.31:'F2',  
            92.50:'F2#',
            98.00:'G2', 
            103.80:'G2#',
            110.00:'A2', 
            116.50:'B2b',
            123.50:'B2', 
            130.80:'C3', 
            138.60:'C3#',
            146.80:'D3',  
            155.60:'E3b', 
            164.80:'E3',  
            174.60:'F3',  
            185.00:'F3#',
            196.00:'G3',
            207.70:'G3#',
            220.00:'A3',
            233.10:'B3b',
            246.90:'B3', 
            261.60:'C4', 
            277.20:'C4#',
            293.70:'D4', 
            311.10:'E4b', 
            329.60:'E4', 
            349.20:'F4', 
            370.00:'F4#',
            392.00:'G4',
            415.30:'G4#',
            440.00:'A4',
            466.20:'B4b',
            493.90:'B4', 
            523.30:'C5', 
            554.40:'C5#',
            587.30:'D5', 
            622.30:'E5b', 
            659.30:'E5', 
            698.50:'F5', 
            740.00:'F5#',
            784.00:'G5',
            830.60:'G5#',
            880.00:'A5',
            932.30:'B5b',
            987.80:'B5', 
            1047.00:'C6',
            1109.0:'C6#',
            1175.0:'D6', 
            1245.0:'E6b', 
            1319.0:'E6', 
            1397.0:'F6', 
            1480.0:'F6#',
            1568.0:'G6',
            1661.0:'G6#',
            1760.0:'A6',
            1865.0:'B6b',
            1976.0:'B6', 
            2093.0:'C7'
            } 
            
class NoteTrainer(object):  
        
    def main(self):

        
        # Build frequency, noteName dictionary
        tunerNotes = build_default_tuner_range()

        # Sort the keys and turn into a numpy array for logical indexing
        frequencies = numpy.array(sorted(tunerNotes.keys()))

        top_note = len(tunerNotes)-1
        bot_note = 0
        

        top_note = 24
        bot_note = 0
        
        # Misc variables for program controls
        inputnote = 1                               # the y value on the plot
        signal_level = 0                              # volume level
        trys = 1
        soundgate = 19                             # zero is loudest possible input level
        targetnote = 0
        SR = SoundRecorder()                          # recording device (usb mic)
        
        c=[]
        while trys < 20:
            
            SR.setup()
            raw_data_signal = SR.getAudio()                                         #### raw_data_signal is the input signal data 
            signal_level = round(abs(loudness(raw_data_signal)),2)                  #### find the volume from the audio sample
            
            try: 
                inputnote = round(freq_from_autocorr(raw_data_signal,SR.RATE),2)    #### find the freq from the audio sample
                
            except:
                inputnote == 0
                
            SR.close()
            
            if inputnote > frequencies[len(tunerNotes)-1]:                        #### not interested in notes above the notes list
                continue
                
            if inputnote < frequencies[0]:                                     #### not interested in notes below the notes list
                continue    
                    
            if signal_level > soundgate:                                        #### basic noise gate to stop it guessing ambient noises 
                continue      
            
            targetnote = closest_value_index(frequencies, round(inputnote, 2))      #### find the closest note in the keyed array
            c.append(targetnote)
            trys += 1
        return c
            ##### use the controls to make changes to the data #####           
            
#if __name__ == '__main__':
w= NoteTrainer().main()
print w

[48, 50, 48, 48, 48, 48, 50, 48, 43, 48, 50, 52, 52, 52, 48, 50, 53, 50, 52]


In [16]:
from midiutil.MidiFile import MIDIFile

def makemidi(array):
    mf =MIDIFile(1)
    track = 0
    channel = 0
    volume = 100
    time=0
    mf.addTrackName(track, time, "Sample Track")
    mf.addTempo(track, time, 120)
    for note in array:
        pitch=note+36
        duration=1
        mf.addNote(track, channel, pitch, time, duration, volume)
        time += 2
    return mf

xx=makemidi(w)

# write it to disk
with open("Desktop/output.mid", 'wb') as outf:
    xx.writeFile(outf)

In [17]:
import music21
score = music21.converter.parse('Desktop/output.mid')
#key = score.analyze('key')
key1 = score.analyze('Krumhansl')
print(key1.tonic.name, key1.mode)

('C', 'major')


In [21]:
from mingus.core import progressions, intervals
from mingus.core import chords as ch
from mingus.containers import NoteContainer, Note
from mingus.midi import fluidsynth
import time
import sys
from random import random, choice, randrange
SF2 = 'Desktop/Pizz Violins.sf2'
prog=["vi", "ii", "iii7",  "I7", "viidom7", "iii7", "V7"]
progression = ["IIm7", "IIm7", "IIm7", "IIm7"]

# progression = ["I", "vi", "ii", "iii7",               "I7", "viidom7", "iii7",
# "V7"]

key = key1.tonic.name

# If True every second iteration will be played in double time, starting on the
# first

#double_time = choice([True, False, True])
double_time = False

orchestrate_second = True
swing = True
play_solo = True
play_drums = True
play_bass = True
play_chords = True
bar_length = 1.75
song_end = 1.1

# Control beginning of solos and chords

solo_start = 8
solo_end = 20
chord_start = 16
chord_end = 24

# Channels

chord_channel = 1
chord_channel2 = 7
chord_channel3 = 3
bass_channel = 6
solo_channel = 13
random_solo_channel = False
if not fluidsynth.init(SF2):
    print "Couldn't load soundfont", SF2
    sys.exit(1)
chords = progressions.to_chords(progression, key)
loop = 1
while loop < 7:
    for p in progs:
        
    i = 0
    if random_solo_channel:
        solo_channel = choice(range(5, 8) + [11])
    for chord in chords:
        c = NoteContainer(chords[i])
        l = Note(c[0].name)
        n = Note('C')
        l.octave_down()
        l.octave_down()
        print ch.determine(chords[i])[0]

        if not swing and play_chords and loop > chord_start and loop\
             < chord_end:
            fluidsynth.play_NoteContainer(c, chord_channel, randrange(50, 75))
        if play_chords and loop > chord_start and loop < chord_end:
            if orchestrate_second:
                if loop % 2 == 0:
                    fluidsynth.play_NoteContainer(c, chord_channel2,
                            randrange(50, 75))
            else:
                fluidsynth.play_NoteContainer(c, chord_channel2, randrange(50,
                        75))

        if double_time:
            beats = [random() > 0.5 for x in range((loop % 2 + 3) * 8)]
        else:
            beats = [random() > 0.5 for x in range(8)]
        t = 0
        for beat in beats:

            # Play random note

            if beat and play_solo and loop > solo_start and loop < solo_end:
                fluidsynth.stop_Note(n)
                if t % 2 == 0:
                    n = Note(choice(c).name)
                elif random() > 0.5:
                    if random() < 0.46:
                        n = Note(intervals.third(choice(c).name, key))
                    elif random() < 0.46:
                        n = Note(intervals.seventh(choice(c).name, key))
                    else:
                        n = Note(choice(c).name)
                    if t > 0 and t < len(beats) - 1:
                        if beats[t - 1] and not beats[t + 1]:
                            n = Note(choice(c).name)
                fluidsynth.play_Note(n, solo_channel, randrange(80, 110))
                print n

            # Repeat chord on half of the bar

            if play_chords and t != 0 and loop > chord_start and loop\
                 < chord_end:
                if swing and random() > 0.95:
                    fluidsynth.play_NoteContainer(c, chord_channel3,
                            randrange(20, 75))
                elif t % (len(beats) / 2) == 0 and t != 0:
                    fluidsynth.play_NoteContainer(c, chord_channel3,
                            randrange(20, 75))

            # Play bass note

            if play_bass and t % 4 == 0 and t != 0:
                l = Note(choice(c).name)
                l.octave_down()
                l.octave_down()
                fluidsynth.play_Note(l, bass_channel, randrange(50, 75))
            elif play_bass and t == 0:
                fluidsynth.play_Note(l, bass_channel, randrange(50, 75))

            # Drums

            if play_drums and loop > 0:
                if t % (len(beats) / 2) == 0 and t != 0:
                    fluidsynth.play_Note(Note('E', 2), 9, randrange(50, 100))  # snare
                else:
                    if random() > 0.8 or t == 0:
                        fluidsynth.play_Note(Note('C', 2), 9, randrange(20,
                                100))  # bass
                if t == 0 and random() > 0.75:
                    fluidsynth.play_Note(Note('C#', 3), 9, randrange(60, 100))  # crash
                if swing:
                    if random() > 0.9:
                        fluidsynth.play_Note(Note('A', 2), 9, randrange(50,
                                100))  # hihat open
                    elif random() > 0.6:
                        fluidsynth.play_Note(Note('G#', 2), 9, randrange(100,
                                120))  # hihat closed
                    if random() > 0.95:
                        fluidsynth.play_Note(Note('E', 2), 9, 100)  # snare
                elif t % 2 == 0:
                    fluidsynth.play_Note(Note('A#', 2), 9, 100)  # hihat open
                else:
                    if random() > 0.9:
                        fluidsynth.play_Note(Note('E', 2), 9, 100)  # snare
            if swing:
                if t % 2 == 0:
                    time.sleep((bar_length / (len(beats) * 3)) * 4)
                else:
                    time.sleep((bar_length / (len(beats) * 3)) * 2)
            else:
                time.sleep(bar_length / len(beats))
            t += 1
        fluidsynth.stop_NoteContainer(c, chord_channel)
        fluidsynth.stop_NoteContainer(c, chord_channel2)
        fluidsynth.stop_NoteContainer(c, chord_channel3)
        fluidsynth.stop_Note(l, bass_channel)
        fluidsynth.stop_Note(n, solo_channel)
        i += 1
    print '-' * 20
    loop += 1

D minor seventh
D minor seventh
D minor seventh
D minor seventh
--------------------
D minor seventh
D minor seventh
D minor seventh
D minor seventh
--------------------
D minor seventh
D minor seventh
D minor seventh
D minor seventh
--------------------
D minor seventh
D minor seventh
D minor seventh
D minor seventh
--------------------
D minor seventh
D minor seventh
D minor seventh
D minor seventh
--------------------
D minor seventh
D minor seventh
D minor seventh
D minor seventh
--------------------
